In [59]:
import pandas as pd
import requests
import tweepy
import yaml 
import os
import json
import numpy as np

In [37]:
tweets = api.search_users(q="Chris Bassolino")

In [40]:
len(tweets)

1

In [38]:
tweets[0].screen_name

'marketectured'

# Read PitchBook data in
## Start with companies that HAVE received VC funding. These are the 1 companies (VC investment =1)

In [127]:
vc_general_info_df = pd.read_excel("../data/raw/CA_VC_PitchBook/Company_General_Information.xlsx",header=6)
vc_general_info_df.head(2)
# interesting columns= Company ID (primary key), Description, Company Name, HQ Post Code, Primary Industry Code, Primary Contact, Year Founded

,Company ID,Company Name,HQ Post Code,Description,Primary Industry Code,HQ Location,Ownership Status,Company Financing Status,Active Investors,Website,HQ Email,HQ Phone,Primary Contact,Primary Contact Title,Primary Contact Email,Primary Contact Phone,Year Founded,PitchBook Link
0,61743-97,ZypMedia,94102,Developer of an online advertisement platform ...,Social/Platform Software,"San Francisco, CA",Privately Held (backing),Venture Capital-Backed,"Sinclair Broadcast Group, Sinclair Digital Ven...",www.zypmedia.com,info@extend.tv,+1 (415) 894-9181,Mark Goldman,"Co-Founder, Board Member & Chief Executive Off...",mgoldman@extend.tv,+1 (415) 894-9181,2013.0,NaN
1,103815-64,Zype,10001,Developer of a cloud platform for video publis...,Social/Platform Software,"New York, NY",Privately Held (backing),Venture Capital-Backed,"Alpine Meridian, Berlin Ventures, Bill Dolan, ...",www.zype.com,contact@zype.com,NaN,Chris Bassolino,"Co-Founder, Chief Operating Officer & Director",NaN,NaN,2013.0,NaN


In [125]:
vc_general_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 18 columns):
Company ID                  1023 non-null object
Company Name                1022 non-null object
HQ Post Code                925 non-null object
Description                 1000 non-null object
Primary Industry Code       1000 non-null object
HQ Location                 998 non-null object
Ownership Status            1000 non-null object
Company Financing Status    1000 non-null object
Active Investors            966 non-null object
Website                     983 non-null object
HQ Email                    706 non-null object
HQ Phone                    705 non-null object
Primary Contact             980 non-null object
Primary Contact Title       980 non-null object
Primary Contact Email       871 non-null object
Primary Contact Phone       723 non-null object
Year Founded                970 non-null float64
PitchBook Link              0 non-null float64
dtypes: float64(2), obj

In [130]:
vc_last_financing_df = pd.read_excel("../data/raw/CA_VC_PitchBook/Last_Financing_Details.xlsx",header=6)
vc_last_financing_df .head(2)
# interesting columns = Company ID ( primary key), Company Name, Growth Rate, Size Multiple, last financing date,
# last financing Size, Last financing valuation, Last Financing Deal Type 2

# Note : Only want series A or later, filter OUT the seed rounds

,Company ID,Company Name,Revenue,Growth Rate,Size Multiple,Last Financing Date,Last Financing Deal Type,Last Financing Deal Type 2,Last Financing Size,Last Financing Valuation,...,Ownership Status,Company Financing Status,Financing Status Note,Active Investors,Acquirers,Primary Industry Code,Description,HQ Location,Website,PitchBook Link
0,61743-97,ZypMedia,NaN,0.000000,0.144068,2015-04-24,Early Stage VC,Series B,4.42,9.47,...,Privately Held (backing),Venture Capital-Backed,The company raised $4.42 million of Series B v...,"Sinclair Broadcast Group, Sinclair Digital Ven...",NaN,Social/Platform Software,Developer of an online advertisement platform ...,"San Francisco, CA",www.zypmedia.com,NaN
1,103815-64,Zype,NaN,0.919952,9.045216,2016-08-01,Seed Round,Seed,3.60,10.05,...,Privately Held (backing),Venture Capital-Backed,The company raised $3.6 million of seed ventur...,"Alpine Meridian, Berlin Ventures, Bill Dolan, ...",NaN,Social/Platform Software,Developer of a cloud platform for video publis...,"New York, NY",www.zype.com,NaN


In [131]:
vc_last_financing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 21 columns):
Company ID                     1023 non-null object
Company Name                   1022 non-null object
Revenue                        113 non-null float64
Growth Rate                    957 non-null float64
Size Multiple                  957 non-null float64
Last Financing Date            869 non-null datetime64[ns]
Last Financing Deal Type       994 non-null object
Last Financing Deal Type 2     585 non-null object
Last Financing Size            695 non-null float64
Last Financing Valuation       352 non-null float64
Last Financing Status          994 non-null object
Ownership Status               1000 non-null object
Company Financing Status       1000 non-null object
Financing Status Note          997 non-null object
Active Investors               966 non-null object
Acquirers                      1 non-null object
Primary Industry Code          1000 non-null object
Descriptio

In [135]:
vc_company_financials_df = pd.read_excel("../data/raw/CA_VC_PitchBook/Public_Company_Financials.xlsx",header=6)
vc_company_financials_df.head(2)
# Interesting columns are NOTHING

,Company ID,Company Name,Exchange,Ticker,Fiscal Period,Revenue,Gross Profit,EBITDA,Net Income,Enterprise Value,Ownership Status,Company Financing Status,Last Financing Date,Last Financing Deal Type,Last Financing Size,PitchBook Link
0,61743-97,ZypMedia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Privately Held (backing),Venture Capital-Backed,2015-04-24,Early Stage VC,4.42,NaN
1,103815-64,Zype,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Privately Held (backing),Venture Capital-Backed,2016-08-01,Seed Round,3.60,NaN


In [134]:
vc_company_financials_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 16 columns):
Company ID                  1023 non-null object
Company Name                1022 non-null object
Exchange                    1 non-null object
Ticker                      1 non-null object
Fiscal Period               122 non-null object
Revenue                     113 non-null float64
Gross Profit                7 non-null float64
EBITDA                      19 non-null float64
Net Income                  28 non-null float64
Enterprise Value            2 non-null float64
Ownership Status            1000 non-null object
Company Financing Status    1000 non-null object
Last Financing Date         869 non-null datetime64[ns]
Last Financing Deal Type    994 non-null object
Last Financing Size         695 non-null float64
PitchBook Link              0 non-null float64
dtypes: datetime64[ns](1), float64(7), object(8)
memory usage: 128.2+ KB


In [138]:
vc_social_web_df =  pd.read_excel("../data/raw/CA_VC_PitchBook/Social_and_Web_Presence.xlsx",header=6)
vc_social_web_df.head(2)
# interesting columns = company id (primary key), company name, growth rate, size multiple, majestic referring domains
# facebook likes, Tiwtter followers, Employees, Total raised

,Company ID,Company Name,Growth Rate,Size Multiple,Compete Unique Visitors,Majestic Referring Domains,Facebook Likes,Twitter Followers,Employees,Total Raised,Primary Industry Code,Description,Active Investors,Company Financing Status,Ownership Status,HQ Location,Website,PitchBook Link
0,61743-97,ZypMedia,0.000000,0.144068,NaN,NaN,NaN,51.0,12.0,6.82,Social/Platform Software,Developer of an online advertisement platform ...,"Sinclair Broadcast Group, Sinclair Digital Ven...",Venture Capital-Backed,Privately Held (backing),"San Francisco, CA",www.zypmedia.com,NaN
1,103815-64,Zype,0.919952,9.045216,NaN,188.0,1904.0,8352.0,11.0,3.60,Social/Platform Software,Developer of a cloud platform for video publis...,"Alpine Meridian, Berlin Ventures, Bill Dolan, ...",Venture Capital-Backed,Privately Held (backing),"New York, NY",www.zype.com,NaN


In [137]:
vc_social_web_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 18 columns):
Company ID                    1023 non-null object
Company Name                  1022 non-null object
Growth Rate                   957 non-null float64
Size Multiple                 957 non-null float64
Compete Unique Visitors       0 non-null float64
Majestic Referring Domains    881 non-null float64
Facebook Likes                731 non-null float64
Twitter Followers             806 non-null float64
Employees                     839 non-null float64
Total Raised                  818 non-null float64
Primary Industry Code         1000 non-null object
Description                   1000 non-null object
Active Investors              966 non-null object
Company Financing Status      1000 non-null object
Ownership Status              1000 non-null object
HQ Location                   998 non-null object
Website                       983 non-null object
PitchBook Link                0

# Join these four dataframes into one

In [318]:
vc_general_info_colDrop_df = vc_general_info_df[["Company ID", "Description", "Company Name", "HQ Post Code", "Primary Industry Code",
                                         "Primary Contact", "Year Founded"]]

In [319]:
vc_last_financing_colDrop_df =vc_last_financing_df[["Company ID","Company Name", "Growth Rate", "Size Multiple",
                                    "Last Financing Date","Last Financing Size","Last Financing Valuation",
                                                   "Last Financing Deal Type 2 "]]

In [320]:
vc_social_web_colDrop_df  =vc_social_web_df [["Company ID", "Company Name", "Growth Rate", 
                                              "Size Multiple", "Majestic Referring Domains",
        "Facebook Likes", "Twitter Followers", "Employees", "Total Raised"]]

In [321]:
final_vc_df = vc_general_info_colDrop_df.merge(vc_last_financing_colDrop_df, on='Company ID').merge(vc_social_web_colDrop_df, 
                                                                                     on='Company ID')

In [322]:
final_vc_df .columns

Index(['Company ID', 'Description', 'Company Name_x', 'HQ Post Code',
       'Primary Industry Code', 'Primary Contact', 'Year Founded',
       'Company Name_y', 'Growth Rate_x', 'Size Multiple_x',
       'Last Financing Date', 'Last Financing Size',
       'Last Financing Valuation', 'Last Financing Deal Type 2 ',
       'Company Name', 'Growth Rate_y', 'Size Multiple_y',
       'Majestic Referring Domains', 'Facebook Likes', 'Twitter Followers',
       'Employees', 'Total Raised'],
      dtype='object')

In [323]:
final_vc_df .drop(['Company Name_y',
                  'Growth Rate_y','Size Multiple_y'],axis=1,inplace=True)

In [324]:
final_vc_df.rename(columns={'Growth Rate_x':'Growth Rate',"Size Multiple_x":'Size Multiple',
                           "Company Name_x":"Company Name"},inplace=True) # rename cols

In [325]:
final_vc_df.head(2)

,Company ID,Description,Company Name,HQ Post Code,Primary Industry Code,Primary Contact,Year Founded,Growth Rate,Size Multiple,Last Financing Date,Last Financing Size,Last Financing Valuation,Last Financing Deal Type 2,Company Name,Majestic Referring Domains,Facebook Likes,Twitter Followers,Employees,Total Raised
0,61743-97,Developer of an online advertisement platform ...,ZypMedia,94102,Social/Platform Software,Mark Goldman,2013.0,0.000000,0.144068,2015-04-24,4.42,9.47,Series B,ZypMedia,NaN,NaN,51.0,12.0,6.82
1,103815-64,Developer of a cloud platform for video publis...,Zype,10001,Social/Platform Software,Chris Bassolino,2013.0,0.919952,9.045216,2016-08-01,3.60,10.05,Seed,Zype,188.0,1904.0,8352.0,11.0,3.60


### Filter out the seed rounds

In [326]:
final_vc_df['Last Financing Deal Type 2 '].unique()

array(['Series B', 'Seed', 'Series B2', nan, 'Series A', 'Series B1',
       'Series D', 'Series C', 'Series A1', 'Series E', 'Series A2',
       'Angel', 'Corporate', 'Senior Debt', 'Series 2', 'Series 1',
       'Add-on', 'Loan', 'Series CC', 'Series F', 'Acquisition Financing',
       'Series G', 'Revolving Credit Line', 'Series C3', 'Series 3',
       'Series AA', 'Series B3', 'Subordinated Debt', 'Series C1'], dtype=object)

In [327]:
final_vc_financeTypeFilter_df = final_vc_df.loc[(final_vc_df['Last Financing Deal Type 2 ']!='Seed') & 
                (final_vc_df['Last Financing Deal Type 2 ']!='Angel'),: ]

In [328]:
final_vc_financeTypeFilter_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 844 entries, 0 to 1030
Data columns (total 19 columns):
Company ID                     836 non-null object
Description                    813 non-null object
Company Name                   835 non-null object
HQ Post Code                   760 non-null object
Primary Industry Code          813 non-null object
Primary Contact                799 non-null object
Year Founded                   792 non-null float64
Growth Rate                    780 non-null float64
Size Multiple                  780 non-null float64
Last Financing Date            693 non-null datetime64[ns]
Last Financing Size            558 non-null float64
Last Financing Valuation       292 non-null float64
Last Financing Deal Type 2     398 non-null object
Company Name                   835 non-null object
Majestic Referring Domains     723 non-null float64
Facebook Likes                 601 non-null float64
Twitter Followers              650 non-null float64
Employees  

In [329]:
final_vc_financeTypeFilter_df.head(2)

,Company ID,Description,Company Name,HQ Post Code,Primary Industry Code,Primary Contact,Year Founded,Growth Rate,Size Multiple,Last Financing Date,Last Financing Size,Last Financing Valuation,Last Financing Deal Type 2,Company Name,Majestic Referring Domains,Facebook Likes,Twitter Followers,Employees,Total Raised
0,61743-97,Developer of an online advertisement platform ...,ZypMedia,94102,Social/Platform Software,Mark Goldman,2013.0,0.0,0.144068,2015-04-24,4.42,9.47,Series B,ZypMedia,NaN,NaN,51.0,12.0,6.82
2,60184-09,Developer of non-invasive sensors for laborato...,Zyomed,91011,Diagnostic Equipment,Sandeep Gulati,2007.0,0.0,0.054054,2016-05-17,10.61,132.61,Series B2,Zyomed,2.0,NaN,NaN,NaN,23.11


# Drop companies missing the zip code, year founded, and primary contact
- Can't impute this

In [330]:
final_vc_dropFinanceZipYear_df = final_vc_financeTypeFilter_df.loc[
    (final_vc_financeTypeFilter_df['HQ Post Code'].isnull()==False) &
    (final_vc_financeTypeFilter_df['Year Founded'].isnull()==False) &
    (final_vc_financeTypeFilter_df['Primary Contact'].isnull()==False),: ]

In [331]:
final_vc_dropFinanceZipYear_df .info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 733 entries, 0 to 1021
Data columns (total 19 columns):
Company ID                     733 non-null object
Description                    733 non-null object
Company Name                   733 non-null object
HQ Post Code                   733 non-null object
Primary Industry Code          733 non-null object
Primary Contact                733 non-null object
Year Founded                   733 non-null float64
Growth Rate                    709 non-null float64
Size Multiple                  709 non-null float64
Last Financing Date            635 non-null datetime64[ns]
Last Financing Size            518 non-null float64
Last Financing Valuation       272 non-null float64
Last Financing Deal Type 2     366 non-null object
Company Name                   733 non-null object
Majestic Referring Domains     656 non-null float64
Facebook Likes                 555 non-null float64
Twitter Followers              598 non-null float64
Employees  

In [332]:
final_vc_dropFinanceZipYear_df .describe()

,Year Founded,Growth Rate,Size Multiple,Last Financing Size,Last Financing Valuation,Majestic Referring Domains,Facebook Likes,Twitter Followers,Employees,Total Raised
count,733.000000,709.000000,709.000000,518.000000,272.000000,656.000000,5.550000e+02,598.000000,648.000000,614.000000
mean,2009.028649,0.428218,80.120564,25.845000,237.640882,1941.554878,1.896565e+05,13760.147157,134.146605,62.852785
std,7.209625,1.262782,615.563496,121.737093,1179.235327,17991.973626,1.957759e+06,65343.708872,655.558765,602.658726
min,1903.000000,-6.928121,0.007453,0.010000,0.170000,1.000000,1.000000e+00,1.000000,1.000000,0.020000
25%,2007.000000,0.000000,0.873353,2.000000,16.057500,30.750000,2.150000e+02,216.500000,11.000000,3.440000
50%,2011.000000,0.042641,3.249337,6.000000,41.665000,115.500000,1.135000e+03,1098.000000,27.000000,11.185000
75%,2013.000000,0.511800,12.278546,16.000000,120.392500,424.500000,8.949500e+03,4656.250000,67.000000,36.325000
max,2016.000000,13.422807,13063.853941,2250.000000,18000.000000,440467.000000,4.187224e+07,769227.000000,12000.000000,14688.060000


# Impute missing values
- Growth Rate : impute median
- Size Multiple: imput median
- Last Financing Date : dont't really need this ( only need for angel companies)
- Last Financing Size : impute median
- Last Financing Valuation: Drop this ( too many nulls)
- Majestic Referring Domains: Impute Median
- Facebook Like: Impute median
- Twitter followers: impute emdian
- Employees: impute median
- Total raised: impute median

In [333]:
median_values={}
for row in final_vc_dropFinanceZipYear_df.describe(): # get median values
    median_values[row]=final_vc_dropFinanceZipYear_df.describe()[row]["50%"]

In [334]:
median_values

{'Employees': 27.0,
 'Facebook Likes': 1135.0,
 'Growth Rate': 0.042640929986614687,
 'Last Financing Size': 6.0,
 'Last Financing Valuation': 41.664999999999999,
 'Majestic Referring Domains': 115.5,
 'Size Multiple': 3.2493367722918203,
 'Total Raised': 11.184999999999999,
 'Twitter Followers': 1098.0,
 'Year Founded': 2011.0}

In [347]:
imputed_final_df = final_vc_dropFinanceZipYear_df.copy()
for key in median_values: # update the nan values with the median
    updated_col = final_vc_dropFinanceZipYear_df.loc[:,key].copy()
    updated_col = updated_col.fillna(median_values[key])
    imputed_final_df.loc[:,key] = updated_col


In [348]:
imputed_final_df.head(2)

,Company ID,Description,Company Name,HQ Post Code,Primary Industry Code,Primary Contact,Year Founded,Growth Rate,Size Multiple,Last Financing Date,Last Financing Size,Last Financing Valuation,Last Financing Deal Type 2,Company Name,Majestic Referring Domains,Facebook Likes,Twitter Followers,Employees,Total Raised,VC_invested
0,61743-97,Developer of an online advertisement platform ...,ZypMedia,94102,Social/Platform Software,Mark Goldman,2013.0,0.0,0.144068,2015-04-24,4.42,9.47,Series B,ZypMedia,115.5,1135.0,51.0,12.0,6.82,1
2,60184-09,Developer of non-invasive sensors for laborato...,Zyomed,91011,Diagnostic Equipment,Sandeep Gulati,2007.0,0.0,0.054054,2016-05-17,10.61,132.61,Series B2,Zyomed,2.0,1135.0,1098.0,27.0,23.11,1


### Add a 1 for the column VC_Invested

In [349]:
imputed_final_df['VC_invested']=1

In [350]:
imputed_final_df.head(2)

,Company ID,Description,Company Name,HQ Post Code,Primary Industry Code,Primary Contact,Year Founded,Growth Rate,Size Multiple,Last Financing Date,Last Financing Size,Last Financing Valuation,Last Financing Deal Type 2,Company Name,Majestic Referring Domains,Facebook Likes,Twitter Followers,Employees,Total Raised,VC_invested
0,61743-97,Developer of an online advertisement platform ...,ZypMedia,94102,Social/Platform Software,Mark Goldman,2013.0,0.0,0.144068,2015-04-24,4.42,9.47,Series B,ZypMedia,115.5,1135.0,51.0,12.0,6.82,1
2,60184-09,Developer of non-invasive sensors for laborato...,Zyomed,91011,Diagnostic Equipment,Sandeep Gulati,2007.0,0.0,0.054054,2016-05-17,10.61,132.61,Series B2,Zyomed,2.0,1135.0,1098.0,27.0,23.11,1


In [351]:
imputed_final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 733 entries, 0 to 1021
Data columns (total 20 columns):
Company ID                     733 non-null object
Description                    733 non-null object
Company Name                   733 non-null object
HQ Post Code                   733 non-null object
Primary Industry Code          733 non-null object
Primary Contact                733 non-null object
Year Founded                   733 non-null float64
Growth Rate                    733 non-null float64
Size Multiple                  733 non-null float64
Last Financing Date            635 non-null datetime64[ns]
Last Financing Size            733 non-null float64
Last Financing Valuation       733 non-null float64
Last Financing Deal Type 2     366 non-null object
Company Name                   733 non-null object
Majestic Referring Domains     733 non-null float64
Facebook Likes                 733 non-null float64
Twitter Followers              733 non-null float64
Employees  

# Tweepy to find usernames of the people

In [380]:
def username_search(name):
    """Run a search on twitter for the given name. Returns the first username (should be the most relevant)

    First try searching for the person's name + company. If that does not work, try just searching for the 
    person's name"""
    credentials = yaml.load(open(os.path.expanduser('~/.ssh/api_credentials.yml')))
    auth = tweepy.OAuthHandler(credentials['twitter']['consumer_key'], credentials['twitter']['consumer_secret'],)
    auth.set_access_token(credentials['twitter']['token'], credentials['twitter']['token_secret'])
    api = tweepy.API(auth,wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    tweets = api.search_users(q=name)
    try: # search the name and the company
        return( tweets[0].screen_name)
    except Exception as e: # try just the name
        try:
            first,last,company = name.split(" ")
            tweets = api.search_users(q=first+" "+last)
            return( tweets[0].screen_name)
        except Exception as e:
            print(e)
            return "NaN"

In [381]:
username_search("Chandran Sankaran")

Rate limit reached. Sleeping for: 408


'chandransank'

### Test the accuracy of this api for finding usernames


In [382]:
test_names = ['Jonathan Hilgart','Tim Cook Apple', "Smita Bakshi Zyante","Rob Fuggetta zuberance",
              "Charles Hogan tranzlogic", 
              "Duke Chung travelbank", "Jake Fields treeline","Eric Min Zwift","Dominic Lewis Transifex",
             "Dimitris Glezos Transifex"]
names_username = {"Jonathan Hilgart":"topofthehil",'Tim Cook':"tim_cook", "Smita Bakshi":"sbaksh",
                 "Rob Fuggetta":"robfuggetta", "Charles Hogan":"Tranzlogic",
                 "Duke Chung":"Duke_Chung", "Jake Fields":"JakeFields","Eric Min":"werkdodger",
                  "Dominic Lewis":"domwlewis", " Dominic Lewis":"glezos"}

In [383]:
test_search= {}
for name in test_names:
    result = username_search(name)
    print(result)
    test_search[name]=result

topofthehil
tim_cook
sbaksh
list index out of range
NaN
charliehogan
Duke_Chung
JakeFields
werkdodger
domwlewis
glezos


In [373]:
username_intersection = set(test_search.values()).intersection(set(names_username.values()))

In [374]:
len(username_intersection)/len(names_username.keys())

0.8

In [375]:
test_search

{'Charles Hogan tranzlogic': 'charliehogan',
 'Dimitris Glezos Transifex': 'glezos',
 'Dominic Lewis Transifex': 'domwlewis',
 'Duke Chung travelbank': 'Duke_Chung',
 'Eric Min Zwift': 'werkdodger',
 'Jake Fields treeline': 'JakeFields',
 'Jonathan Hilgart': 'topofthehil',
 'Rob Fuggetta zuberance': 'NaN',
 'Smita Bakshi Zyante': 'sbaksh',
 'Tim Cook Apple': 'tim_cook'}

#### Pretty good! About 80% accuracy in finding twitter handles

### Add the Twitter username to the pandas df

In [379]:
for idx,row in enumerate(imputed_final_df.iterrows()):
    imputed_final_df['Twitter_Username']=username_search(row[1]['Primary Contact'])
    if idx%100 ==0:
        print(f"Finished {idx}")


Finished 0
not enough values to unpack (expected 3, got 2)
not enough values to unpack (expected 3, got 2)
not enough values to unpack (expected 3, got 2)
not enough values to unpack (expected 3, got 2)
not enough values to unpack (expected 3, got 2)
not enough values to unpack (expected 3, got 2)
not enough values to unpack (expected 3, got 2)
not enough values to unpack (expected 3, got 2)
list index out of range
Finished 100
not enough values to unpack (expected 3, got 2)
not enough values to unpack (expected 3, got 2)
not enough values to unpack (expected 3, got 2)
not enough values to unpack (expected 3, got 2)
not enough values to unpack (expected 3, got 2)
not enough values to unpack (expected 3, got 2)
not enough values to unpack (expected 3, got 2)
not enough values to unpack (expected 3, got 2)
not enough values to unpack (expected 3, got 2)
not enough values to unpack (expected 3, got 2)
Finished 200
not enough values to unpack (expected 3, got 2)
not enough values to unpack

RateLimitError: [{'message': 'Rate limit exceeded', 'code': 88}]